# Day 20: Jurassic Jigsaw

[*Advent of Code 2020 day 20*](https://adventofcode.com/2020/day/20) and [*solution megathread*](https://redd.it/kgo01p)

[![nbviewer](https://raw.githubusercontent.com/jupyter/design/master/logos/Badges/nbviewer_badge.svg)](https://nbviewer.jupyter.org/github/UncleCJ/advent-of-code/blob/cj/2020/20/code.ipynb) [![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/UncleCJ/advent-of-code/cj?filepath=2020%2F20%2Fcode.ipynb)

In [1]:
from IPython.display import HTML
import sys
sys.path.append('../../')
import common

downloaded = common.refresh()
%store downloaded >downloaded

Writing 'downloaded' (dict) to file 'downloaded'.


## Part One

In [2]:
HTML(downloaded['part1'])

## Boilerplate

Let's try using [pycodestyle_magic](https://github.com/mattijn/pycodestyle_magic) with pycodestyle (flake8 stopped working for me in VS Code Jupyter). Now how does type checking work?

In [3]:
%load_ext pycodestyle_magic

In [4]:
%pycodestyle_on

In [5]:
testdata = """Tile 2311:
..##.#..#.
##..#.....
#...##..#.
####.#...#
##.##.###.
##...#.###
.#.#.#..##
..#....#..
###...#.#.
..###..###

Tile 1951:
#.##...##.
#.####...#
.....#..##
#...######
.##.#....#
.###.#####
###.##.##.
.###....#.
..#.#..#.#
#...##.#..

Tile 1171:
####...##.
#..##.#..#
##.#..#.#.
.###.####.
..###.####
.##....##.
.#...####.
#.##.####.
####..#...
.....##...

Tile 1427:
###.##.#..
.#..#.##..
.#.##.#..#
#.#.#.##.#
....#...##
...##..##.
...#.#####
.#.####.#.
..#..###.#
..##.#..#.

Tile 1489:
##.#.#....
..##...#..
.##..##...
..#...#...
#####...#.
#..#.#.#.#
...#.#.#..
##.#...##.
..##.##.##
###.##.#..

Tile 2473:
#....####.
#..#.##...
#.##..#...
######.#.#
.#...#.#.#
.#########
.###.#..#.
########.#
##...##.#.
..###.#.#.

Tile 2971:
..#.#....#
#...###...
#.#.###...
##.##..#..
.#####..##
.#..####.#
#..#.#..#.
..####.###
..#.#.###.
...#.#.#.#

Tile 2729:
...#.#.#.#
####.#....
..#.#.....
....#..#.#
.##..##.#.
.#.####...
####.#.#..
##.####...
##..#.##..
#.##...##.

Tile 3079:
#.#.#####.
.#..######
..#.......
######....
####.#..#.
.#...#.##.
#.#####.##
..#.###...
..#.......
..#.###...""".splitlines()

inputdata = downloaded['input'].splitlines()

monster_pattern = """                  # 
#    ##    ##    ###
 #  #  #  #  #  #   """.splitlines()

111:41: W291 trailing whitespace


In [6]:
from math import sqrt
from enum import Enum


def parse(lines):
    blank_lines = [idx for idx, line in enumerate(lines)
                   if line == '']
    boundaries = zip(
        [0] + [idx + 1 for idx in blank_lines],
        blank_lines + [len(lines)]
    )

    tiles = dict()
    for low, high in boundaries:
        tile = Tile(lines[low:high])
        tiles[tile.id] = tile

    return tiles


class Directions(Enum):
    def __repr__(self):
        return self.name

    def flipped(self):
        return Directions((Directions.WEST.value - self.value) % 4)

    def translate(self, flipped, rotation):
        if not flipped:
            return Directions((self.value + rotation) % 4)
        else:
            return Directions((self.flipped().value + rotation) % 4)

    def rotation_to(self, flipped, target_direction):
        if not flipped:
            return (target_direction.value - self.value) % 4
        else:
            return (target_direction.value - self.flipped().value) % 4

    NORTH = 0
    EAST = 1
    SOUTH = 2
    WEST = 3


class Tile(object):
    def __init__(self, lines):
        self.id = int(lines[0][5:-1])
        self.lines = lines[1:]
        self.size = len(self.lines)
        self.values = []
        self.values_flipped = []
        self.is_flipped = False
        self.rotated = 0
        self.set_values()
        self.fits_with = []
        self.fits_with_flipped = []

    def __repr__(self):
        return (f'id: {str(self.id)}, values: {str(self.values)}' +
                f', values_flipped: {str(self.values_flipped)}' +
                f', fits_with: {str(self.fits_with)}' +
                f', fits_with_flipped: {str(self.fits_with_flipped)}'
                )

    def set_values(self):
        self.values = [0] * 4
        self.values_flipped = [0] * 4
        for direction in Directions:
            self.values[direction.value] = self.value(direction)
            self.values_flipped[direction.value] = \
                self.value_flipped(direction)

    def value(self, direction):
        return sum([1 << b for b, c
                    in enumerate(self.get_edge(direction)[::-1])
                    if c == '#'])

    def value_flipped(self, direction):
        return sum([1 << b for b, c
                    in enumerate(self.get_edge(direction))
                    if c == '#'])

    def get_edge(self, direction):
        if direction == Directions.NORTH:
            return self.lines[0]
        elif direction == Directions.EAST:
            return [line[-1] for line in self.lines]
        elif direction == Directions.SOUTH:
            return self.lines[-1][::-1]
        elif direction == Directions.WEST:
            return [line[0] for line in self.lines[::-1]]

    def get_as_placed(self, flipped, rotation):
        rotation = rotation % 4
        output = self.lines[:]

        output_flipped = output[:]
        if flipped:
            for i in range(len(output)):
                output_flipped[i] = ''.join(
                    [line[i] for line in output]
                    )

        output_rotated = output_flipped[:]
        if rotation == 1:
            for i in range(len(output)):
                output_rotated[i] = ''.join(
                    [line[i] for line in output_flipped[::-1]]
                    )
        elif rotation == 2:
            for i in range(len(output)):
                output_rotated[i] = output_flipped[-(1 + i)][::-1]
        elif rotation == 3:
            for i in range(len(output)):
                output_rotated[i] = ''.join(
                    [line[-(1 + i)] for line in output_flipped]
                    )

        return output_rotated

    def fit_with(self, other):
        for i in Directions:
            for j in Directions:
                if (self.values[i.value]
                        == other.values_flipped[j.value]):
                    self.fits_with.append((i, other.id, j))
                    other.fits_with.append((j, self.id, i))
                if (self.values_flipped[i.value]
                        == other.values_flipped[j.value]):
                    self.fits_with_flipped.append((i, other.id, j))
                    other.fits_with_flipped.append((j, self.id, i))

    def translate_fits_with(fits_with, flipped, rotation):
        return [(our_edge.translate(flipped, rotation),
                 other_id,
                 other_edge)
                for our_edge, other_id, other_edge in fits_with]

    def get_fits_with_transated(self, flipped, rotation):
        if not flipped:
            return (Tile.translate_fits_with(self.fits_with,
                                             flipped,
                                             rotation),
                    Tile.translate_fits_with(self.fits_with_flipped,
                                             flipped,
                                             rotation))
        else:
            return (Tile.translate_fits_with(self.fits_with_flipped,
                                             flipped,
                                             rotation),
                    Tile.translate_fits_with(self.fits_with,
                                             flipped,
                                             rotation))


class Solution(object):
    def __init__(self, tiles):
        self.size = round(sqrt(len(tiles)))
        self.tiles = tiles
        self.placements = [[(0, 0, False)
                            for x in range(self.size)]
                           for y in range(self.size)]

    def fit_tiles(self, debug=False):
        corner_ids = []
        tile_list = list(self.tiles.values())
        for i in range(len(tile_list) - 1):
            for j in range(i + 1, len(tile_list)):
                tile_list[i].fit_with(tile_list[j])
        for tile in tile_list:
            if debug:
                print(f'id: {tile.id}, fits_with: {tile.fits_with}' +
                      f', fits_with_flipped: {tile.fits_with_flipped}')
            if len(tile.fits_with) + len(tile.fits_with_flipped) == 2:
                corner_ids.append(tile.id)
        return corner_ids

    def look_direction(self, x, y):
        if x > 0 and self.placements[y][x - 1][0]:
            return (Directions.WEST, x - 1, y)
        elif y > 0 and self.placements[y - 1][x][0]:
            return (Directions.NORTH, x, y - 1)
        elif x < self.size and self.placements[y][x + 1][0]:
            return (Directions.EAST, x + 1, y)
        elif y < self.size and self.placements[y + 1][x][0]:
            return (Directions.SOUTH, x, y + 1)

    def place_by_tile(look_direction, fits_with, flipped):
        for previous_edge, our_id, our_edge in fits_with:
            if previous_edge == look_direction.translate(False, 2):
                our_rotation = our_edge.rotation_to(flipped, look_direction)
                return our_id, flipped, our_rotation
        return False

    def place_by_previous(self, x, y):
        look_direction, look_x, look_y = self.look_direction(x, y)
        previous_id, previous_flipped, previous_rotation \
            = self.placements[look_y][look_x]
        previous_tile = self.tiles[previous_id]
        # print(f'previous_tile: {previous_tile}')
        previous_fits_with, previous_fits_with_flipped \
            = previous_tile.get_fits_with_transated(
                previous_flipped, previous_rotation)
        placement = Solution.place_by_tile(
            look_direction, previous_fits_with, False)
        if not placement:
            placement = Solution.place_by_tile(
                look_direction, previous_fits_with_flipped, True)

        self.placements[y][x] = placement

    def place(self, nw_id, nw_flipped, nw_rotation):
        for x, y in [(x, y)
                     for y in range(self.size)
                     for x in range(self.size)]:
            if x == 0 and y == 0:
                self.placements[y][x] = (nw_id, nw_flipped, nw_rotation)
                continue
            self.place_by_previous(x, y)
        return self.placements

In [7]:
tiles = parse(testdata)
solution = Solution(tiles)
corner_ids = solution.fit_tiles()
print(f'corner tiles: {corner_ids}')
placements = solution.place(1951, True, 3)
print(placements)
# print(tiles[1951].get_as_placed(True, 3))

corner tiles: [1951, 1171, 2971, 3079]
[[(1951, True, 3), (2311, True, 3), (3079, False, 0)], [(2729, True, 3), (1427, True, 3), (2473, True, 2)], [(2971, True, 3), (1489, True, 3), (1171, True, 1)]]


In [8]:
# tiles = parse(inputdata)
# solution = Solution(tiles)
# corner_ids = solution.fit_tiles()
# print(f'corner tiles: {corner_ids}')
# placements = solution.place(1951, True, 3)
# print(placements)
# print(tiles[1951].get_as_placed(True, 3))

In [9]:
HTML(downloaded['part1_footer'])

## Part Two

In [10]:
HTML(downloaded['part2'])

In [11]:
class Mosaic(object):
    def __init__(self, solution):
        self.mosaic_size = solution.size
        self.tiles = solution.tiles
        self.placements = solution.placements

    def get_tiles_as_placed(self):
        output = []
        for placement_line in self.placements:
            output.append(
                [self.tiles[tile_id].get_as_placed(flipped, rotation)
                 for tile_id, flipped, rotation in placement_line]
            )
        return output

    def get_mosaic(self):
        output = []
        for tile_line in self.get_tiles_as_placed():
            output_tile_line = ['']*len(tile_line[0])
            for i in range(len(output_tile_line)):
                output_tile_line[i] = ' '.join(
                    [tile[i] for tile in tile_line]
                )
            output.append('\n'.join(output_tile_line))
        return '\n\n'.join(output)

    def get_trimmed_tiles(self):
        output = []
        for trimmed_tile_line in self.get_tiles_as_placed():
            output_tile_line = []
            for tile in trimmed_tile_line:
                output_tile = []
                for line in tile[1:-1]:
                    output_tile.append(line[1:-1])
                output_tile_line.append(output_tile)
            output.append(output_tile_line)
        return output

    def get_composite(self):
        output = []
        for trimmed_tile_line in self.get_trimmed_tiles():
            output_tile_line = ['']*len(trimmed_tile_line[0])
            for i in range(len(output_tile_line)):
                output_tile_line[i] = ''.join(
                    [trimmed_tile[i] for trimmed_tile in trimmed_tile_line]
                )
            output.append('\n'.join(output_tile_line))
        return '\n'.join(output)

    def find_monsters(self, flipped, rotation):
        composite_tile = Tile(['Tile 0000:']
                              + self.get_composite().splitlines())
        composite = composite_tile.get_as_placed(flipped, rotation)
        monster_idx = [[i for i, ch in enumerate(line) if ch == '#']
                       for line in monster_pattern]
        monsters = 0
        for i in range(len(composite) - len(monster_pattern) + 1):
            for j in range(len(composite) - len(monster_pattern[0]) + 1):
                no_monster = False
                for m_i in range(len(monster_idx)):
                    no_monster = any([True for idx in monster_idx[m_i]
                                      if composite[i + m_i][j + idx] != '#'])
                    if no_monster:
                        break
                if not no_monster:
                    monsters += 1

        roughness = sum([line.count('#') for line in composite])
        roughness -= monsters*sum([len(idx_line) for idx_line in monster_idx])

        return monsters, roughness, flipped, rotation

    def find_any_monsters(self):
        results = []
        for flipped in {True, False}:
            for rotation in range(4):
                results.append(self.find_monsters(flipped, rotation))
        return results

In [12]:
mosaic = Mosaic(solution)
print(mosaic.get_mosaic())
print()
composite_tile = Tile(['Tile 0000:'] + mosaic.get_composite().splitlines())
print('\n'.join(composite_tile.get_as_placed(True, 0)))
print(mosaic.find_any_monsters())

#...##.#.. ..###..### #.#.#####.
..#.#..#.# ###...#.#. .#..######
.###....#. ..#....#.. ..#.......
###.##.##. .#.#.#..## ######....
.###.##### ##...#.### ####.#..#.
.##.#....# ##.##.###. .#...#.##.
#...###### ####.#...# #.#####.##
.....#..## #...##..#. ..#.###...
#.####...# ##..#..... ..#.......
#.##...##. ..##.#..#. ..#.###...

#.##...##. ..##.#..#. ..#.###...
##..#.##.. ..#..###.# ##.##....#
##.####... .#.####.#. ..#.###..#
####.#.#.. ...#.##### ###.#..###
.#.####... ...##..##. .######.##
.##..##.#. ....#...## #.#.#.#...
....#..#.# #.#.#.##.# #.###.###.
..#.#..... .#.##.#..# #.###.##..
####.#.... .#..#.##.. .######...
...#.#.#.# ###.##.#.. .##...####

...#.#.#.# ###.##.#.. .##...####
..#.#.###. ..##.##.## #..#.##..#
..####.### ##.#...##. .#.#..#.##
#..#.#..#. ...#.#.#.. .####.###.
.#..####.# #..#.#.#.# ####.###..
.#####..## #####...#. .##....##.
##.##..#.. ..#...#... .####...#.
#.#.###... .##..##... .####.##.#
#...###... ..##...#.. ...#..####
..#.#....# ##.#.#.... ...##.....

.####..

In [13]:
tiles = parse(inputdata)
solution = Solution(tiles)
corner_ids = solution.fit_tiles()
print(f'corner tiles: {corner_ids}')
placements = solution.place(3187, True, 0)
print(placements)
mosaic = Mosaic(solution)
print(mosaic.get_mosaic())
print()
composite_tile = Tile(['Tile 0000:'] + mosaic.get_composite().splitlines())
print('\n'.join(composite_tile.get_as_placed(True, 0)))
# print(mosaic.find_any_monsters())
print(mosaic.find_monsters(False, 2))

corner tiles: [3187, 2239, 2503, 3851]
[[(3187, True, 0), (2251, False, 3), (1423, True, 1), (3533, True, 2), (2029, True, 1), (1559, True, 1), (1427, False, 0), (1109, False, 3), (3803, True, 3), (2011, False, 0), (2833, True, 1), (3851, False, 1)], [(2861, False, 0), (1889, False, 1), (2039, True, 3), (1733, True, 2), (3833, False, 1), (1667, True, 1), (2531, True, 1), (1847, True, 1), (1051, False, 1), (2879, True, 3), (3923, False, 3), (2339, True, 2)], [(3137, False, 0), (3461, True, 2), (1171, True, 3), (1567, False, 1), (3517, True, 0), (1283, False, 0), (3389, True, 3), (1811, False, 0), (1741, True, 0), (3221, True, 1), (2377, True, 3), (2297, False, 1)], [(3643, False, 3), (2731, False, 3), (1951, True, 2), (1181, True, 2), (2089, False, 0), (3413, False, 1), (1129, False, 0), (1093, True, 0), (1543, False, 3), (2099, True, 3), (3083, True, 2), (1861, True, 0)], [(1019, True, 3), (1787, True, 0), (2789, False, 0), (2137, False, 1), (2113, False, 3), (1307, False, 1), (3767, F

In [14]:
HTML(downloaded['part2_footer'])